In [ ]:
cd $PRS_CODEBASE

## QC target set

#### Prepare pheno files for oncoarray datasets

In [10]:
python -c '
import pandas as pd 
import os


def save_pheno_files_all(df_agg_formatted, super_pop):

    try:
        os.makedirs(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}/impX")
    except:
        pass
    try:
        os.symlink(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}", f"'$PRS_DATASETS'/cimba_{super_pop.lower()}")
    except:
        pass
    df_agg_formatted[df_agg_formatted["super_pop"]==super_pop].\
        loc[:,["FID", "IID", "super_pop", "pop"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}/pop.panel", sep="\t", index=False)
    df_agg_formatted[df_agg_formatted["super_pop"]==super_pop].\
        loc[:,["FID", "IID", "label"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}/pheno", sep="\t", index=False)
    df_agg_formatted[df_agg_formatted["super_pop"]==super_pop].\
        loc[:,["FID", "IID", "label", "censage", "diagnosis_age"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}/cov2", sep="\t", index=False)
    df_agg_formatted[df_agg_formatted["super_pop"]==super_pop].\
        loc[:,["FID", "IID", "censage"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}/pheno_age", sep="\t", index=False)


def save_pheno_files_by_brca(df_agg_formatted, super_pop, brca_type):
    
    try:
        os.makedirs(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}_brca{brca_type}/impX")
    except:
        pass
    try:
        os.symlink(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}_brca{brca_type}", f"'$PRS_DATASETS'/cimba_{super_pop.lower()}_brca{brca_type}")
    except:
        pass
    
    df_agg_formatted[(df_agg_formatted["brca_type"]==brca_type) & (df_agg_formatted["super_pop"]==super_pop)].\
        loc[:,["FID", "IID", "super_pop", "pop"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}_brca{brca_type}_oncoarray/pop.panel", sep="\t", index=False)
    df_agg_formatted[(df_agg_formatted["brca_type"]==brca_type) & (df_agg_formatted["super_pop"]==super_pop)].\
        loc[:,["FID", "IID", "label"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}_brca{brca_type}_oncoarray/pheno", sep="\t", index=False)
    df_agg_formatted[(df_agg_formatted["brca_type"]==brca_type) & (df_agg_formatted["super_pop"]==super_pop)].\
        loc[:,["FID", "IID", "label", "censage", "diagnosis_age"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}_brca{brca_type}_oncoarray/cov2", sep="\t", index=False)
    df_agg_formatted[(df_agg_formatted["brca_type"]==brca_type) & (df_agg_formatted["super_pop"]==super_pop)].\
        loc[:,["FID", "IID", "censage"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}_brca{brca_type}_oncoarray/pheno_age", sep="\t", index=False)

# Merge raw tables
df2=pd.read_csv("'$PRS_DATASETS_ELKON'/cimba/impX/raw/pheno_data/B2_ONCO_Phenotype_04_07_2023.txt", encoding="iso-8859-1")
df1=pd.read_csv("'$PRS_DATASETS_ELKON'/cimba/impX/raw/pheno_data/B1_ONCO_Phenotype_04_07_2023.txt", encoding="iso-8859-1")
df_agg=pd.concat((df2,df1))

# Censor cases where mastectomy or oophorectomy were performed after diagnosis
df_agg=df_agg[~(((df_agg["ocage"]< df_agg["bpoophage"]) | (df_agg["bc1age"]< df_agg["bpmage"])))]

# Add diagnosis age (either BC or OC)
df_agg["diagnosis_age"]= df_agg[["bc1age", "bc2age", "ocage"]].min(axis=1) 
# Assign age of dignosis 200 for NA 
df_agg["diagnosis_age"].fillna(200, inplace=True)

#Assign super_pop and pop values
df_agg["pop"]=df_agg["Country"]
ethnicities={"1": "EUR", "2": "AFR", "3": "EAS", "4": "HIS", "5": "AJ", "6": "EUR"}
df_agg["super_pop"]=df_agg["ethnicity"].apply(lambda a: ethnicities.get(a, "other_unknown"))
df_agg["pop"]=df_agg[["super_pop", "Country"]].apply(lambda a: "AJ" if a["super_pop"]=="AJ" else a["Country"].replace(" ","_"), axis=1)



# Cases are samples that has "diagnosis_age". NAs are control
df_agg["label"]=df_agg["diagnosis_age"].apply(lambda a: 1 if a==200 else 2)
df_agg.to_csv("'$PRS_DATASETS_ELKON'/cimba/impX/raw/pheno_data/ONCO_Phenotype_04_07_2023.tsv", sep="\t", index=False)

# Leave only relevant fields and rename them properly
df_agg_formatted=df_agg.loc[pd.isnull(df_agg["ocage"]),["Onc_ID","Onc_ID", "super_pop", "pop", "Mut1Gene", "diagnosis_age", "censage", "label"]]
df_agg_formatted.columns = ["FID", "IID", "super_pop", "pop", "brca_type", "diagnosis_age", "censage", "label"] 

save_pheno_files_all(df_agg_formatted, "AJ")
save_pheno_files_all(df_agg_formatted, "EUR")

save_pheno_files_by_brca(df_agg_formatted, "AJ", 1)
save_pheno_files_by_brca(df_agg_formatted, "EUR", 1)
save_pheno_files_by_brca(df_agg_formatted, "AJ", 2)
save_pheno_files_by_brca(df_agg_formatted, "EUR", 2)

print("Done!")
'



Done!


In [ ]:
function stats () {
    pop=$1
    brca_type=$2
    if [[ -z $brca_type ]]; then 
        brca_suffix=""
        printf "${pop}\tall\t\t"
      
    else
        brca_suffix="_brca${brca_type}"
        printf "${pop}\t${brca_type}\t\t"
    fi
    controls=$(cat $PRS_DATASETS/cimba_${pop}${brca_suffix}/pheno | cut -f 3 | grep 1 | wc -l )
    cases=$(cat  $PRS_DATASETS/cimba_${pop}${brca_suffix}/pheno | cut -f 3 | grep 2 | wc -l )
    total=$(cat  $PRS_DATASETS/cimba_${pop}${brca_suffix}/pheno | cut -f 3 | wc -l )
    echo -e "${cases}\t${controls}\t${total}"
#   echo -e "n_cases=${cases}\tn_control=${control}\ttotal=${total}"
}
echo -e "pop\tbrca_type\tn_cases\tn_ctrl\ttotal"
echo "=== all ==="
stats "eur"
stats "aj"

echo "=== by brca ==="
stats "eur" "1"
stats "eur" "2"
stats "aj" "1"
stats "aj" "2"


#### Prepare pheno files for icogs dataset

In [7]:
python -c '
import pandas as pd 
import os


def save_pheno_files_all(df_agg_formatted, super_pop):

    try:
        os.makedirs(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}/impX_gen")
    except:
        pass
    try:
        os.symlink(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}", f"'$PRS_DATASETS'/cimba_{super_pop.lower()}")
    except:
        pass
    df_agg_formatted[df_agg_formatted["super_pop"]==super_pop].\
        loc[:,["FID", "IID", "super_pop", "pop"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}/pop.panel", sep="\t", index=False)
    df_agg_formatted[df_agg_formatted["super_pop"]==super_pop].\
        loc[:,["FID", "IID", "label"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}/pheno", sep="\t", index=False)
    df_agg_formatted[df_agg_formatted["super_pop"]==super_pop].\
        loc[:,["FID", "IID", "label", "diagnosis_age"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}/cov", sep="\t", index=False)
    df_agg_formatted[df_agg_formatted["super_pop"]==super_pop].\
        loc[:,["FID", "IID", "censage"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}/pheno_age", sep="\t", index=False)


def save_pheno_files_by_brca(df_agg_formatted, super_pop, brca_type):
    
    try:
        os.makedirs(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}_brca{brca_type}_icogs/impX_gen")
    except:
        pass
    try:
        os.symlink(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}_brca{brca_type}_icogs", f"'$PRS_DATASETS'/cimba_{super_pop.lower()}_brca{brca_type}")
    except:
        pass
    df_agg_formatted[(df_agg_formatted["brca_type"]==brca_type) & (df_agg_formatted["super_pop"]==super_pop)].\
        loc[:,["FID", "IID", "super_pop", "pop"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}_brca{brca_type}_icogs/pop.panel", sep="\t", index=False)
    df_agg_formatted[(df_agg_formatted["brca_type"]==brca_type) & (df_agg_formatted["super_pop"]==super_pop)].\
        loc[:,["FID", "IID", "label"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}_brca{brca_type}_icogs/pheno", sep="\t", index=False)
    df_agg_formatted[(df_agg_formatted["brca_type"]==brca_type) & (df_agg_formatted["super_pop"]==super_pop)].\
        loc[:,["FID", "IID", "label", "diagnosis_age"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}_brca{brca_type}_icogs/cov2", sep="\t", index=False)
    df_agg_formatted[(df_agg_formatted["brca_type"]==brca_type) & (df_agg_formatted["super_pop"]==super_pop)].\
        loc[:,["FID", "IID", "censage"]].to_csv(f"'$PRS_DATASETS_ELKON'/cimba_{super_pop.lower()}_brca{brca_type}_icogs/pheno_age", sep="\t", index=False)

# Merge raw tables
df1=pd.read_csv("'$PRS_DATASETS_ELKON'/cimba/impX/raw/pheno_data/B1_iCOGS_Phenotype_04_08_2023.txt", encoding="iso-8859-1")
df2=pd.read_csv("'$PRS_DATASETS_ELKON'/cimba/impX/raw/pheno_data/B2_iCOGS_Phenotype_04_08_2023.txt", encoding="iso-8859-1")
df_agg=pd.concat((df1,df2))

# Censor cases where mastectomy or oophorectomy were performed after diagnosis
df_agg=df_agg[~(((df_agg["ocage"]< df_agg["bpoophage"]) | (df_agg["bc1age"]< df_agg["bpmage"])))]

# Add diagnosis age (either BC or OC)
df_agg["diagnosis_age"]= df_agg[["bc1age", "bc2age", "ocage"]].min(axis=1) 
# Assign age of dignosis 200 for NA 
df_agg["diagnosis_age"].fillna(200, inplace=True)

#Assign super_pop and pop values
df_agg["pop"]=df_agg["Country"]
ethnicities={"1": "EUR", "2": "AFR", "3": "EAS", "4": "HIS", "5": "AJ", "6": "EUR"}
df_agg["ethnicity"]=df_agg["ethnicity"].apply(lambda a: str(int(a)) if type(a)==float and not pd.isnull(a) else a)
df_agg["super_pop"]=df_agg["ethnicity"].apply(lambda a: ethnicities.get(a, "other_unknown"))
df_agg["pop"]=df_agg[["super_pop", "Country"]].apply(lambda a: "AJ" if a["super_pop"]=="AJ" else a["Country"].replace(" ","_"), axis=1)



# Cases are samples that has "diagnosis_age". NAs are control
df_agg["label"]=df_agg["diagnosis_age"].apply(lambda a: 1 if a==200 else 2)
df_agg.to_csv("'$PRS_DATASETS_ELKON'/cimba/impX/raw/pheno_data/iCOGS_Phenotype_04_08_2023.tsv", sep="\t", index=False)

# Leave only relevant fields and rename them properly
df_agg_formatted=df_agg.loc[pd.isnull(df_agg["ocage"]),["SG_ID","SG_ID", "super_pop", "pop", "Mut1Gene", "diagnosis_age", "censage", "label"]]
df_agg_formatted.columns = ["FID", "IID", "super_pop", "pop", "brca_type", "diagnosis_age", "censage", "label"] 

save_pheno_files_all(df_agg_formatted, "AJ")
save_pheno_files_all(df_agg_formatted, "EUR")

save_pheno_files_by_brca(df_agg_formatted, "AJ", 1)
save_pheno_files_by_brca(df_agg_formatted, "EUR", 1)
save_pheno_files_by_brca(df_agg_formatted, "AJ", 2)
save_pheno_files_by_brca(df_agg_formatted, "EUR", 2)

print("Done!")
'



Done!


In [ ]:
while true; do
    echo 
    sleep 60
done &

function split_cimba_to_subsets () {
    pop=$1
    array=$2
    brca_type=$3
    if [[ -z $brca_type ]]; then 
        brca_suffix=""
        echo "pop=${pop}, brca_type=all"

    else
        brca_suffix="_brca${brca_type}"
        echo "pop=${pop}, brca_type=${brca_type}"
    fi
   
    plink2 --bfile ${PRS_DATASETS_ELKON}/cimba/impX_gen/raw/bed/brca${brca_type}_${array}_all --keep ${PRS_DATASETS_ELKON}/cimba_${pop}${brca_suffix}_${array}/pheno \
    --make-bed --out ${PRS_DATASETS_ELKON}/cimba_${pop}${brca_suffix}_${array}/impX_gen/ds
}

echo "=== all ==="
split_cimba_to_subsets "aj" "icogs" "1"
split_cimba_to_subsets "aj" "icogs" "2"
split_cimba_to_subsets "eur" "icogs" "1"
split_cimba_to_subsets "eur" "icogs" "2"

split_cimba_to_subsets "aj" "oncoarray" "1"
split_cimba_to_subsets "aj" "oncoarray" "2"
split_cimba_to_subsets "eur" "oncoarray" "1"
split_cimba_to_subsets "eur" "oncoarray" "2"


In [ ]:

function run_qc_target () {
target=$1
imp=$2
    bash qc_target_data.sh --target ${target} --imp ${imp}
}

cd $PRS_CODEBASE
run_qc_target "cimba_aj_brca2" "impX"



In [ ]:
while true; do
    echo 
    sleep 60
done &
function run_calc_prs_mono () {

    
    cd $PRS_CODEBASE
    target=$1
    imp=$2
        bash calc_prs_pt3.sh --discovery bcac_onco_eur-5pcs --target ${target} --imp ${imp} --stage 3 
}

cd $PRS_CODEBASE
run_calc_prs_mono "cimba_aj_brca1" "impX"

# run_calc_prs_mono "cimba_aj_brca1" "impX"

In [ ]:
run_calc_prs_mono "cimba_aj_brca1" "impX"

In [ ]:
cd /specific/netapp5/gaga/gaga-pd/prs_data/datasets/dec/cimba/impX/raw/bed
plink --bfile  brca1_chr1_fixed1 --merge-list brca1_list.txt --allow-no-sex  --make-bed --out ds_brca1_fixed && \
plink --bfile  ds_brca1_fixed --update-chr updated_chrs 1 2 --make-bed --out ds_brca1_chr_fixed 
plink --bfile  brca2_chr1_fixed2 --merge-list brca2_list.txt --allow-no-sex  --make-bed --out ds_brca2_fixed $$ \
plink --bfile  ds_brca2_fixed --update-chr updated_chrs 1 2 --make-bed --out ds_brca2_chr_fixed

#### Create SNP map files

In [ ]:
for chr in {1..22}; do cat  $PRS_DATASETS/cimba/impX_gen/raw/data/onco_brca1_info_chr${chr}_varid.txt | grep rs | awk '{if (NR==1){next}; print $5"\t"substr($5,1,index($5,":")-1)}'; done > $PRS_DATASETS/cimba/impX_gen/raw/brca1_oncoarray_snps_map
for chr in {1..22}; do cat  $PRS_DATASETS/cimba/impX_gen/raw/data/onco_brca2_info_chr${chr}_varid.txt | grep rs | awk '{if (NR==1){next}; print $5"\t"substr($5,1,index($5,":")-1)}'; done > $PRS_DATASETS/cimba/impX_gen/raw/brca2_oncoarray_snps_map
for chr in {1..22}; do cat  $PRS_DATASETS/cimba/impX_gen/raw/data/icogs_brca1_info_chr${chr}_varid.txt | grep rs | awk '{if (NR==1){next}; print $5"\t"substr($5,1,index($5,":")-1)}'; done > $PRS_DATASETS/cimba/impX_gen/raw/brca1_icogs_snps_map
for chr in {1..22}; do cat  $PRS_DATASETS/cimba/impX_gen/raw/data/icogs_brca2_info_chr${chr}_varid.txt | grep rs | awk '{if (NR==1){next}; print $5"\t"substr($5,1,index($5,":")-1)}'; done > $PRS_DATASETS/cimba/impX_gen/raw/brca2_icogs_snps_map


#### Create POS map files

In [ ]:
for chr in {1..22}; do cat  $PRS_DATASETS/cimba/impX_gen/raw/data/onco_brca1_info_chr${chr}_varid.txt | grep rs | awk '{if (NR==1){next}; print $6"\t"substr($5,1,index($5,":")-1)}'; done > $PRS_DATASETS/cimba/impX_gen/raw/brca1_oncoarray_pos_map
for chr in {1..22}; do cat  $PRS_DATASETS/cimba/impX_gen/raw/data/onco_brca2_info_chr${chr}_varid.txt | grep rs | awk '{if (NR==1){next}; print $6"\t"substr($5,1,index($5,":")-1)}'; done > $PRS_DATASETS/cimba/impX_gen/raw/brca2_oncoarray_pos_map
for chr in {1..22}; do cat  $PRS_DATASETS/cimba/impX_gen/raw/data/icogs_brca1_info_chr${chr}_varid.txt | grep rs | awk '{if (NR==1){next}; print $6"\t"substr($5,1,index($5,":")-1)}'; done > $PRS_DATASETS/cimba/impX_gen/raw/brca1_icogs_pos_map
for chr in {1..22}; do cat  $PRS_DATASETS/cimba/impX_gen/raw/data/icogs_brca2_info_chr${chr}_varid.txt | grep rs | awk '{if (NR==1){next}; print $6"\t"substr($5,1,index($5,":")-1)}'; done > $PRS_DATASETS/cimba/impX_gen/raw/brca2_icogs_pos_map

#### Remove duplicate SNPs from map file

In [ ]:
function remove_dup_snps_from_map(){
 
    brca_type=$1
    array=$2
    map_type=$3
    awk '{if(NR==FNR){c[$1]++; next;} if(c[$2]==0 && $2!=""){print $0}}' \
    <(cat $PRS_DATASETS/cimba/impX_gen/raw/brca${brca_type}_${array}_${map_type} | cut -f 2 | sort | uniq -d) \
    <(cat $PRS_DATASETS/cimba/impX_gen/raw/brca${brca_type}_${array}_${map_type}) \
    > $PRS_DATASETS/cimba/impX_gen/raw/brca${brca_type}_${array}_${map_type}_no_dups
    
    echo "Done $brca_type $array $map_type"

}

remove_dup_snps_from_map 2 icogs snps_map &
remove_dup_snps_from_map 1 icogs snps_map & 
remove_dup_snps_from_map 2 oncoarray snps_map & 
remove_dup_snps_from_map 1 oncoarray snps_map & 
 

#### Create final datasets:
- Fix SNP ids
- Split according to population (AJ vs. EUR), brca type, array 

In [ ]:
function generate_final_sets(){
    pop=$1
    brca_type=$2
    array=$3
    
    
    plink --bfile  /specific/netapp5/gaga/gaga-pd/prs_data/datasets/dec/cimba/impX_gen/raw/bed/brca${brca_type}_${array}_all \
    --update-name  $PRS_DATASETS/cimba/impX_gen/raw/brca${brca_type}_${array}_snps_map_no_dups \
    --keep ${PRS_DATASETS_ELKON}/cimba_${pop}_brca${brca_type}_${array}/pheno \
    --make-bed --out ${PRS_DATASETS_ELKON}/cimba_${pop}_brca${brca_type}_${array}/impX_gen/ds 
}

generate_final_sets aj 1 icogs 

generate_final_sets eur 2 icogs 
generate_final_sets eur 1 icogs 
generate_final_sets eur 2 oncoarray  
generate_final_sets eur 1 oncoarray 


generate_final_sets aj 1 icogs 
generate_final_sets aj 2 icogs 
generate_final_sets aj 1 oncoarray 
generate_final_sets aj 2 oncoarray


In [ ]:
function link_to_gaga(){
    pop=$1
    brca_type=$2
    array=$3
    
    target=cimba_${pop}_brca${brca_type}_${array}
    imp=impX_gen

    ln -s ${PRS_DATASETS_ELKON}/cimba_${pop}_brca${brca_type}_${array} ${PRS_DATASETS}/cimba_${pop}_brca${brca_type}_${array}
 
}

link_to_gaga eur 1 icogs 
link_to_gaga eur 2 icogs 
link_to_gaga eur 1 oncoarray 
link_to_gaga eur 2 oncoarray

link_to_gaga aj 1 icogs 
link_to_gaga aj 2 icogs 
link_to_gaga aj 1 oncoarray 
link_to_gaga aj 2 oncoarray


In [ ]:
function qc_target_data(){
    cd $PRS_CODEBASE
    
    pop=$1
    brca_type=$2
    array=$3
    
    target=cimba_${pop}_brca${brca_type}_${array}
    imp=impX_gen

    
    bash qc_target_data.sh --target ${target} --imp ${imp}   
}
qc_target_data aj 1 oncoarray


qc_target_data eur 1 icogs 
qc_target_data eur 2 icogs 
qc_target_data eur 1 oncoarray 
qc_target_data eur 2 oncoarray

qc_target_data aj 1 icogs 
qc_target_data aj 2 icogs 
qc_target_data aj 1 oncoarray 
qc_target_data aj 2 oncoarray


In [ ]:
function calc_prs_pt3(){
    cd $PRS_CODEBASE
    
    pop=$1
    brca_type=$2
    array=$3
    target=cimba_${pop}_brca${brca_type}_${array}


    discovery=bcac_onco_eur-5pcs # bca_313 # bcac_onco_eur-5pcs
    imp=impX_gen
    bash calc_prs_pt3.sh --discovery ${discovery} --target ${target} --imp ${imp} --stage 3      

#     discovery=bca_313 # bcac_onco_eur-5pcs
#     imp=impX_313
#     bash calc_prs_313.sh --discovery ${discovery} --target ${target} --imp ${imp} --stage 2  
}

calc_prs_pt3 eur 1 oncoarray


calc_prs_pt3 eur 1 icogs 
calc_prs_pt3 eur 2 icogs 
calc_prs_pt3 eur 1 oncoarray 
calc_prs_pt3 eur 2 oncoarray

calc_prs_pt3 aj 1 icogs 
calc_prs_pt3 aj 2 icogs 
calc_prs_pt3 aj 1 oncoarray 
calc_prs_pt3 aj 2 oncoarray


#### Generate 313 genotype sets

In [9]:
python -c '
import pandas as pd 
array="oncoarray"
df1=pd.read_csv(f"'$PRS_DATASETS'/cimba/impX_gen/raw/bed/brca1_{array}_all.bim", sep="\t", header=None)
df2=pd.read_csv("'$PRS_GWASS'/bca_313/313_rsids.tsv",sep="\t")
df_m=pd.merge(df1,df2, left_on=[0,3], right_on=["Chromosome", "Position"], how="right")
df_m=df_m[((df_m[5]==df_m["Reference Allele"]) & (df_m[4]==df_m["EffectAllele"])) | ((df_m[4]==df_m["Reference Allele"]) & (df_m[5]==df_m["EffectAllele"]))]
df_m.loc[:,[1, "SNP"]].to_csv(f"'$PRS_DATASETS'/cimba/impX_gen/raw/313_snps_{array}_map", index=False, header=False, sep="\t")
'

In [5]:
function generate_313_set(){
    pop=$1
    brca_type=$2
    array=$3
    
    mkdir -p ${PRS_DATASETS_ELKON}/cimba_${pop}_brca${brca_type}_${array}/impX_313 || true
    
    plink --bfile  ${PRS_DATASETS_ELKON}/cimba/impX_gen/raw/bed/brca${brca_type}_${array}_all \
    --update-name  ${PRS_DATASETS_ELKON}/cimba/impX_gen/raw/313_snps_${array}_map \
    --keep ${PRS_DATASETS_ELKON}/cimba_${pop}_brca${brca_type}_${array}/pheno \
    --make-bed --out ${PRS_DATASETS_ELKON}/cimba_${pop}_brca${brca_type}_${array}/impX_313/ds 
}

generate_313_set aj 1 oncoarray


PLINK v1.90b6.24 64-bit (6 Jun 2021)           www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /specific/elkon/hagailevi/PRS/datasets/dec/cimba_eur_brca2_icogs/impX_313/ds.log.
Options in effect:
  --bfile /specific/elkon/hagailevi/PRS/datasets/dec/cimba/impX_gen/raw/bed/brca2_icogs_all
  --keep /specific/elkon/hagailevi/PRS/datasets/dec/cimba_eur_brca2_icogs/pheno
  --make-bed
  --out /specific/elkon/hagailevi/PRS/datasets/dec/cimba_eur_brca2_icogs/impX_313/ds
  --update-name /specific/elkon/hagailevi/PRS/datasets/dec/cimba/impX_gen/raw/313_snps_icogs_map

1019915 MB RAM detected; reserving 509957 MB for main workspace.
20672854 variants loaded from .bim file.
1077 people (0 males, 1077 females) loaded from .fam.
--update-name: 322 values updated.
--keep: 641 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 641 founders and 0 nonfounders present.
Calculating 

In [ ]:
for a in {1..22}; do echo "chr ${a}";  cat onco_brca2_info_chr${a}_varid.txt | awk '{if (substr($5,1,2)=="rs"){split($5,a,":"); print a[1]"\t"$6}}'>> /specific/netapp5/gaga/gaga-pd/prs_data/datasets/dec/cimba/impX/raw/brca2_positions; done



function map_positions(){
    pop=$1
    brca_type=$2
    
    if [[ -z $brca_type ]]; then 
        brca_suffix=""
        echo "pop=${pop}, brca_type=all"

    else
        brca_suffix="_brca${brca_type}"
        echo "pop=${pop}, brca_type=${brca_type}"
    fi
    
    for ext in bim bed fam; do 
        mv $PRS_DATASETS/cimba_${pop}${brca_suffix}/impX/ds.${ext} $PRS_DATASETS/cimba_${pop}${brca_suffix}/impX/ds0.${ext}
        mv $PRS_DATASETS/cimba_${pop}${brca_suffix}/impX/ds.QC.${ext} $PRS_DATASETS/cimba_${pop}${brca_suffix}/impX/ds0.QC.${ext}
    done    
    
    
    plink --bfile  $PRS_DATASETS/cimba_${pop}${brca_suffix}/impX/ds0 \
    --update-map  /specific/netapp5/gaga/gaga-pd/prs_data/datasets/dec/cimba/impX/raw/brca2_positions  \
    --make-bed --out $PRS_DATASETS/cimba_${pop}${brca_suffix}/impX/ds
}


map_positions "aj" "1"

map_positions "aj" "2"


In [ ]:
function generate_genotype_set (){
    pop=$1
    brca_type=$2
    
    plink --bfile $PRS_DATASETS/cimba/impX/raw/bed/ds_brca${brca_type}_chr_fixed --keep $PRS_DATASETS/cimba_${pop}_brca${brca_type}/pheno --make-bed --out $PRS_DATASETS/cimba_${pop}_brca${brca_type}/impX/ds
} 

generate_genotype_set "eur" "2"

In [ ]:
function merge_brca_pheno_sets (){
    pop=$1

    for fname in "pop.panel" "cov2" "pheno"; do 
        cat $PRS_DATASETS/cimba_${pop}_brca1/${fname} > $PRS_DATASETS/cimba_${pop}/${fname}
        tail -n +2 $PRS_DATASETS/cimba_${pop}_brca2/${fname} >> $PRS_DATASETS/cimba_${pop}/${fname}
    done
    
merge_brca_pheno_sets "eur"
merge_brca_pheno_sets "aj"

In [ ]:
function merge_brca_genotype_sets (){
    pop=$1

    plink --bfile $PRS_DATASETS/cimba_${pop}_brca1/impX/ds --bmerge $PRS_DATASETS/cimba_${pop}_brca2/impX/ds --allow-no-sex --make-bed --out $PRS_DATASETS/cimba_${pop}/impX/ds
    
merge_brca_pheno_sets "eur"
merge_brca_pheno_sets "aj"

## Generate bem files from gen

#### Reformat sample files according to the oxford (gen) format expected by `plink`

In [ ]:
function reformat_sample_files(){
    target="cimba"
    imp="impX_gen"
    brca_type=$1
    array=$2
    
    echo -e  "ID_1 ID_2 missing sex\n0 0 0 D" > $PRS_DATASETS_ELKON/${target}/${imp}/raw/data/sample_files/brca${brca_type}_${array}.sample
    cat $PRS_DATASETS_ELKON/${target}/${imp}/raw/data/215_brca${brca_type}_${array}_sample_order.txt | awk '{print $1" "$1" 0 2"}' >> $PRS_DATASETS_ELKON/${target}/${imp}/raw/data/sample_files/brca${brca_type}_${array}.sample
}

reformat_sample_files 1 oncoarray
reformat_sample_files 2 oncoarray
reformat_sample_files 1 icogs
reformat_sample_files 2 icogs



#### Reformat oxford (gen+sample) files to bed files using `plink`

In [ ]:
function reformat_gen_to_bed(){
    target="cimba"
    imp="impX_gen"
    brca_type=$1
    array=$2
 
    target_path_elkon="/specific/elkon/hagailevi/PRS/datasets/dec/"${target}"/${imp}/"
    mkdir -p ${target_path_elkon}/raw/bed || echo ""

    for a in {1..22}; do 
        plink --gen $PRS_DATASETS/cimba/impX_gen/raw/data/215_brca${brca_type}_${array}_imputed_probs_chr${a}.gen.gz \
        --sample $PRS_DATASETS/cimba/impX_gen/raw/data/sample_files/brca${brca_type}_${array}.sample \
        --oxford-single-chr ${a} --make-bed --threads 50 \
        --out ${target_path_elkon}/raw/bed/chr${a}_brca${brca_type}_${array}; 
    done
}

reformat_gen_to_bed 1 oncoarray
reformat_gen_to_bed 2 oncoarray
reformat_gen_to_bed 1 icogs
reformat_gen_to_bed 2 icogs

#### Create list files for merge 

In [ ]:
function create_list_file_for_merge(){
    brca_type=$1
    array=$2
    rm $PRS_DATASETS_ELKON/cimba/impX_gen/raw/bed/brca${brca_type}_${array}_list
    for a in {2..22}; do echo "$PRS_DATASETS_ELKON/cimba/impX_gen/raw/bed/chr${a}_brca2_icogs"; done >> $PRS_DATASETS_ELKON/cimba/impX_gen/raw/bed/brca2_icogs_list
}

create_list_file_for_merge 1 oncoarray
create_list_file_for_merge 2 oncoarray
create_list_file_for_merge 1 icogs
create_list_file_for_merge 2 icogs

#### Merge bed files

In [ ]:
function merge_bed_files(){ 
    brca_type=$1
    array=$2
    plink --bfile $PRS_DATASETS_ELKON/cimba/impX_gen/raw/bed/chr1_brca${brca_type}_${array} \
    --merge-list $PRS_DATASETS_ELKON/cimba/impX_gen/raw/bed/brca${brca_type}_${array}_list \
    --make-bed --out $PRS_DATASETS_ELKON/cimba/impX_gen/raw/bed/brca${brca_type}_${array}_all
}

merge_bed_files 2 icogs


merge_bed_from_gen_files 1 oncoarray
merge_bed_from_gen_files 2 oncoarray
merge_bed_from_gen_files 1 icogs
merge_bed_from_gen_files 2 icogs